In [ ]:
import os
from datetime import date, datetime
import yaml
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from experiment.toolkits.plotting_helper import *

project_dir = Path().resolve().parents[1]
sys.path.insert(0, str(project_dir))
import pickle
import datetime
import matplotlib.ticker as ticker

import experiment.toolkits.figure_formatting.formatter_2023_az as fmt
import experiment.toolkits.figure_formatting.production_quality_plots as prod_plot

In [ ]:
home = os.path.expanduser("~") 
os.chdir(str(project_dir))

# Waveform Loading Timing Benchmark

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/wfm-streaming-benchmarking/timing-test"
def get_write_time (wfps, cb, nr):
    if (cb):
        command = base_cmd + " -wfps " + str(wfps) + " -cb " + " -nr " + str(nr)
    else:
        command = base_cmd + " -wfps " + str(wfps) + " -nr " + str(nr)
    return os.popen(command).read()

In [ ]:
reps = 10000
waveforms_per_segment = [1,2,4,8,16,32,64,128] 
continous_buffer = False
time_data  = []

for i in range(len(waveforms_per_segment)):
    print(f"{i}/{len(waveforms_per_segment)}")
    write_time_list = []
    wfps = waveforms_per_segment[i]
    write_time_str = get_write_time(wfps,
                                    continous_buffer, reps)
    print (write_time_str)
    time_data += (float(write_time_str.split("\n")[0]), float(write_time_str.split("\n")[1]))

# Save results

In [ ]:
# Saving the results 
file_name = "timing-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": waveforms_per_segment,
            "continous_buffer": continous_buffer,
            "experiment": file_name,
            "data": time_data
        }, file
        )

# Load results

In [ ]:
load_latest = False 
if not load_latest:
    file_name = "timing-2024-06-13, 09-24-41.pickle"

with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "rb") as file:
    data = pickle.load(file)
    waveforms_per_segment = data["xvals"]
    time_data = data["data"]
    title = data["experiment"]

In [ ]:
def xticks(ax, lst):
    ax.set_xticks(lst)
    diff = (lst[-1] - lst[0]) * 0.05
    ax.set_xlim(lst[0] - diff, lst[-1] + diff)
def yticks(ax, lst):
    ax.set_yticks(lst)
    diff = (lst[-1] - lst[0]) * 0.05
    ax.set_ylim(lst[0] - diff, lst[-1] + diff)


In [ ]:
xvals = waveforms_per_segment
yvals = [(time_data[i] * 1e6) for i in range(0, len(time_data), 2)]
yerr = [(time_data[i] * 1e6) for i in range(1, len(time_data), 2)]
runtimevals = [(xvals[i]*10) for i in range(0, len(xvals))]
runtimevals_err = [0 for i in range(0, len(xvals))]
fig, ax = plt.subplots() 
ax.set_xlabel("Waveforms per segment")
ax.set_ylabel("Time (us)")


prod_plot.production_quality_scatter(fig, ax, [xvals, xvals], [yvals, runtimevals], yerr_list=[yerr, runtimevals_err], save_path=str(project_dir) + "/wfm-loading.pdf")
ax.legend(["Upload", "Stream"],fontsize=4)
yticks(ax, [0, 250, 500,750, 1000])
xticks(ax, [0, 4,8,16,32,64])
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%d'))
plt.savefig("wfm-loading.pdf", format='pdf')
plt.show()

# Waveform Streaming Null Benchmark

In [ ]:
base_cmd = str(project_dir) + "/bin/modules/wfm-streaming-benchmarking/null-test"
def get_write_time (wfps, nm, nr):
    command = base_cmd + " " + str(nm) + " " + str(wfps) + " " + str(nr)
    return os.popen(command).read()

In [ ]:
reps = 1000
waveforms_per_segment = [8,16,32,64] 
number_of_moves = [x for x in range(8, 1505, 8)]
time_data = []

for i in range(len(waveforms_per_segment)):
    print(f"{i}/{len(waveforms_per_segment)}")
    write_time_list = []
    wfps = waveforms_per_segment[i]
    temp_data = []
    for nm in number_of_moves:
        write_time_str = get_write_time(wfps, nm,reps)
        temp_data.append(float(write_time_str))
    time_data.append(temp_data)

# Save results

In [ ]:
# Saving the results 
file_name = "null-test-" + datetime.datetime.now().strftime("%Y-%m-%d, %H-%M-%S.pickle")
with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "wb") as file:
    pickle.dump(
        {
            "xvals": waveforms_per_segment,
            "experiment": file_name,
            "data": time_data
        }, file
        )

# Load results

In [ ]:
load_latest = False 
if not load_latest:
    file_name = "null-test-2024-06-17, 20-32-12.pickle"

with open(os.path.join(home, "LLRS", "resources", "wfm-streaming-characterization", file_name), "rb") as file:
    data = pickle.load(file)
    waveforms_per_segment = data["xvals"]
    results = data["data"]
    title = data["experiment"]

In [ ]:
xvals = waveforms_per_segment
means = [np.mean(results[i]) for i in range(len(results))]
stds = [np.std(results[i]) for i in range(len(results))]
fig, ax = plt.subplots()
ax.set_xlabel("Waveforms per segment")
ax.set_ylabel("Mean probablity of failure")


prod_plot.production_quality_scatter(fig, ax, xvals, means, yerr_list=stds, save_path=str(project_dir) + "/wfm-loading-null.pdf")
yticks(ax, [0,0.2,0.4,0.6,0.8, 1])
xticks(ax, [0,8,16,32,64])
ax.legend(["98.93%", "97.50%", "0.39%", "0.13%"],fontsize=4)
plt.savefig("wfm-loading-null.pdf", format='pdf')
plt.show()